# Statistical Language Modeling

- Language Understanding Systems
- Evgeny A. Stepanov
- stepanov.evgeny.a@gmail.com

This notebook is part of the Laboratory Work for [Language Understanding Systems class](http://disi.unitn.it/~riccardi/page7/page13/page13.html) of [University of Trento](https://www.unitn.it/en).
Laboratory has been ported to jupyer notebook format for remote teaching during [COVID-19 pandemic](https://en.wikipedia.org/wiki/2019%E2%80%9320_coronavirus_pandemic).

Dan Jurafsky and James H. Martin's __Speech and Language Processing__ ([3rd ed. draft](https://web.stanford.edu/~jurafsky/slp3/)) is advised for reading. 

- Section *Corpora and Counting* covers some concepts of *Chapter 2: "Regular Expressions, Text Normalization, Edit Distance"*.
- Sections *Ngrams and Ngram Probabilities* and *Language Models* roughly cover *Chapter 3: "N-gram Language Models"*. 

__Requirements__

- [NL2SparQL4NLU](https://github.com/esrel/NL2SparQL4NLU) dataset

    - run `git clone https://github.com/esrel/NL2SparQL4NLU.git`
    

## 1. Corpora and Counting

### 1.1. Corpus

[Corpus](https://en.wikipedia.org/wiki/Text_corpus) is a collection of written or spoken texts that is used for language research. Before doing anything with a corpus we need to know its properties:

__Corpus Properties__:
- *Format* -- how to read/load it?
- *Language* -- which tools/models can I use?
- *Annotation* -- what it is intended for?
- *Split* for __Evaluation__: (terminology varies from source to source)

| Set         | Purpose                                       |
|:------------|:----------------------------------------------|
| Training    | training model, extracting rules, etc.        |
| Development | tuning, optimization, intermediate evaluation |
| Test        | final evaluation (remains unseen)             |


#### NL2SparQL4NLU

- __Format__:

    - Utterance (sentence) per line
    - Tokenized
    - Lowercased

- __Language__: English monolingual

- __Annotation__: None (for now)

- __Split__: training & test sets

#### Exercise

- define a function to load a corpus into a list-of-lists

- load `NL2SparQL4NLU/dataset/NL2SparQL4NLU.train.utterances.txt`
- print first `2` words (tokens) of the first `10` sentences


In [1]:
trn='NL2SparQL4NLU/dataset/NL2SparQL4NLU.train.utterances.txt'
tst='NL2SparQL4NLU/dataset/NL2SparQL4NLU.test.utterances.txt'

### 1.2. Corpus Descriptive Statistics (Counting)

*Corpus* description in terms of:

- total number of words
- total number of utterances


#### Exercise

- define a function to compute corpus descriptive statistics -- total utterance and word counts.
- compute the statistics for the __training__ and __test__ sets of NL2SparQL4NLU dataset. 
- compare the computed statistics with the reference values below.


| Metric           | Train  | Test   |
|------------------|-------:|-------:|
| Total Words      | 21,453 |  7,117 |
| Total Utterances |  3,338 |  1,084 |


### 1.3. Lexicon

[Lexicon](https://en.wikipedia.org/wiki/Lexicon) is the *vocabulary* of a language. In linguistics, a lexicon is a language's inventory of lexemes.

Linguistic theories generally regard human languages as consisting of two parts: a lexicon, essentially a catalog of a language's words; and a grammar, a system of rules which allow for the combination of those words into meaningful sentences. 

*Lexicon (or Vocabulary) Size* is one of the statistics reported for corpora. While *Word Count* is the number of __tokens__, *Lexicon Size* is the number of __types__ (unique words).


#### Exercise

- define a function to compute a lexicon from corpus in a list-of-lists format
    - sort the list alphabetically
    
- compute the lexicon of the training set of NL2SparQL4NLU dataset
- compare the its size to the reference value below.

| Metric       | Value |
|--------------|------:|
| Lexicon Size | 1,729 |


### 1.4. Frequency List

In Natural Language Processing (NLP), [a frequency list](https://en.wikipedia.org/wiki/Word_lists_by_frequency) is a sorted list of words (word types) together with their frequency, where frequency here usually means the number of occurrences in a given corpus, from which the rank can be derived as the position in the list.

What is a "word"?

- case sensitive counts
- case insensitive counts (our corpus is lowercased)

#### Exercise

- define a function to compute a frequency list for a corpus
- compute frequency list for the training set of NL2SparQL4NLU dataset
- report `5` most frequent words (use can use provided `nbest` function to get a dict of top N items)
- compare the frequencies to the reference values below

| Word   | Frequency |
|--------|----------:|
| the    |     1,337 |
| movies |     1,126 |
| of     |       607 |
| in     |       582 |
| movie  |       564 |


In [2]:
def nbest(d, n=1):
    """
    get n max values from a dict
    :param d: input dict (values are numbers, keys are stings)
    :param n: number of values to get (int)
    :return: dict of top n key-value pairs
    """
    return dict(sorted(d.items(), key=lambda item: item[1], reverse=True)[:n])

### 1.5. Lexicon Operations

It is common to process the lexicon according to the task at hand (not every transformation makes sense for all tasks). The common operations are removing words by frequency (minimum or maximum, i.e. *Frequency Cut-Off*) and removing words for a specific lists (i.e. *Stop Word Removal*).

In computing, [stop words](https://en.wikipedia.org/wiki/Stop_words) are words which are filtered out before or after processing of natural language data (text). Though "stop words" usually refers to the most common words in a language, there is no single universal list of stop words used by all natural language processing tools, and indeed not all tools even use such a list. Some tools specifically avoid removing these stop words to support phrase search.

Any group of words can be chosen as the stop words for a given purpose.

#### Exercises

##### Frequency Cut-Off

- define a function to compute a lexicon from a frequency list applying minimum and maximum frequency cut-offs

    - use default values for min and max
    
- using frequency list for the training set of NL2SparQL4NLU dataset
    
    - compute lexicon applying:
    
        - minimum cut-off 2 (remove words that appear less than 2 times, i.e. remove [hapax legomena](https://en.wikipedia.org/wiki/Hapax_legomenon))
        - maximum cut-off 100 (remove words that appear more that 100 times)
        - both minimum and maximum thresholds together
        
    - report size for each comparing to the reference values in the table

| Operation  | Min | Max | Size |
|------------|----:|----:|-----:|
| original   | N/A | N/A | 1729 |
| cut-off    |   2 | N/A |  950 |
| cut-off    | N/A | 100 | 1694 |
| cut-off    |   2 | 100 |  915 |


##### Stop Word Removal

- define a function to read/load a list of words in token-per-line format (i.e. lexicon)
- load stop word list from `NL2SparQL4NLU/extras/english.stop.txt`
- using Python's built it `set` [methods](https://docs.python.org/2/library/stdtypes.html#set):
    
    - define a function to compute overlap of two lexicons
    - define a function to apply a stopword list to a lexicon

- compare the 100 most frequent words in frequency list of the training set to the list of stopwords (report count)
- apply stopword list to the lexicon of the training set
- report size of the resulting lexicon comparing to the reference values.

| Operation       | Size |
|-----------------|-----:|
| original        | 1729 |
| no stop words   | 1529 |
| top 100 overlap |   50 |

In [3]:
swl='NL2SparQL4NLU/extras/english.stop.txt'

## 2. Ngrams and Ngram Probabilities

[n-gram](https://en.wikipedia.org/wiki/N-gram) is a contiguous sequence of *n* items from a given sequence of text or speech. An n-gram model models sequences, notably natural languages, using the statistical properties of n-grams.

__Example__:

- character n-grams: cat
- word n-grams: the cat is fat


|                     | 1-gram  | 2-gram  | 3-gram  |
|---------------------|---------|---------|---------|
|                     | unigram | bigram  | trigram |
| *Markov Order*      | 0       | 1       | 2       |
| *Character N-grams* | `['c', 'a', 't']` | `['ca', 'at']` | `['cat']` |
| *Word N-grams*      | `['the', 'cat', 'is' , 'fat']` | `['the cat', 'cat is', ...]` | `['the cat is', ...]` |


### 2.1. Counting Ngrams

*Frequency List* of a corpus is essentially a unigram count. Ngram count only differs in a unit of counting -- sequence of $n$ of words. We can compute bigram count by taking sequences of 2 items, trigrams - 3, etc.

#### 2.1.1. Data Structures
For *Frequency List* we have used [`dict`](https://docs.python.org/2/library/stdtypes.html#dict) to store counts.
In ngram counting scenario we still can use dictionary, and use ngram string keys; however, there are better data structures for the task. The frequent data structures used to store ngram counts or probabilities are [hash table](https://en.wikipedia.org/wiki/Hash_table), [trie](https://en.wikipedia.org/wiki/Trie), or [finite state automaton](https://en.wikipedia.org/wiki/Deterministic_acyclic_finite_state_automaton). The pros and cons of each data structure are out of the scope of this lab; for the discussion on efficient data structures for language modeling please refer to [KenLM paper](https://kheafield.com/papers/avenue/kenlm.pdf).

#### 2.1.2. Implementing Trie
There are plenty of implementation of the Trie data structure (e.g. [`pygtrie`](https://github.com/google/pygtrie/) from Google). However, it is simple enough. 

For understanding purposes let's implement our own, such that:

- works on lists of words
- stores in a node: 
    - word
    - its count
    - children (next words)
- implements methods to:
    - add a sequence (updating counts)
    - get a node by a sequence (i.e. prefix)
    - traverse a trie and get all sequences
        - allows to traverse children of any node
    - compute size of ngram vocabulary (V)
    
It is convenient to introduce an `oov` node to the Trie to easily handle sequences not in it.
Thus, we also add `self.ovv` (with `count` set to $0$).

In [2]:
class Node(object):

    def __init__(self, word=None):
        self.word = word
        self.children = {}
        self.count = 0

    def __set__(self, instance, value):
        self.instance = value

    def __get__(self, instance, owner):
        return self.instance


class Trie(object):

    def __init__(self):
        self.root = Node('*')  # trie root
        self.oov = Node()      # node for oov values
        self.size = 0          # depth of trie

    def __set__(self, instance, value):
        self.instance = value

    def __get__(self, instance, owner):
        return self.instance

    def add(self, sequence):
        node = self.root
        node.count += 1  # total word count
        for word in sequence:
            node.children[word] = node.children.setdefault(word, Node(word))
            node = node.children[word]
            node.count += 1

    def get(self, sequence):
        node = self.root
        for word in sequence:
            node = node.children.get(word, self.oov)
        return node

    def traverse(self, node=None, sequence=None, size=None):
        sequence = sequence if sequence else []
        node = self.root if not node else node

        if not node.children:
            yield sequence

        if size:
            if len(sequence) == size:
                yield sequence

        for word, n in node.children.items():
            sequence.append(word)
            yield from self.traverse(n, sequence, size=size)
            sequence.pop()

    def v(self, size=None):
        return len(list(self.traverse(size=size)))

##### Testing out the implementation...

In [5]:
counts = Trie()

# adding 4-grams
counts.add(['a', 'b', 'c', 'd'])
counts.add(['a', 'e', 'c', 'f'])
counts.add(['a', 'b', 'g', 'h'])
counts.add(['x', 'y', 'z', 'a'])

# setting & getting meta-info
counts.size = 4
print('ngram size:', counts.size)

# testing counts for n-grams of various sizes
tests = [['a'], ['a', 'b'], ['a', 'x'], ['e', 'c', 'f'], ['a', 'e', 'c', 'f']]

# getting counts
for seq in tests:
    print(counts.get(seq).count, seq)

# traversing trie: getting all strings
print("All In Trie:")
for seq in counts.traverse():
    print(seq)

# traversing trie: by prefix (['a'])
print("In Trie for: {}".format(['a']))
for seq in counts.traverse(node=counts.get(['a']), sequence=['a']):
    print(counts.get(seq).count, seq)
    
# getting size of ngram vocabulary
for i in range(counts.size):
    print("{}-gram V: {}".format(i+1, counts.v(size=i+1)))

ngram size: 4
3 ['a']
2 ['a', 'b']
0 ['a', 'x']
0 ['e', 'c', 'f']
1 ['a', 'e', 'c', 'f']
All In Trie:
['a', 'b', 'c', 'd']
['a', 'b', 'g', 'h']
['a', 'e', 'c', 'f']
['x', 'y', 'z', 'a']
In Trie for: ['a']
1 ['a', 'b', 'c', 'd']
1 ['a', 'b', 'g', 'h']
1 ['a', 'e', 'c', 'f']
1-gram V: 6
2-gram V: 7
3-gram V: 8
4-gram V: 8


#### Exercise

- define a function to extract ngrams (variable $n$) from a sequence (sentence) represented as a list
- define a function to compute ngram counts from a corpus (list-of-lists) and store as a Trie
- compute bigram counts for the training set of NL2SparQL4NLU
- report `5` most frequent bigrams comparing to the reference values below (you can use `nbest` on `node.children`)


 word 1 | word 2 | count 
:-------|:-------|-------:
show    | me     |   377
the     | movie  |   267
of      | the    |   186
me      | the    |   122
is      | the    |   120


### 2.2. Computing Ngram Probabilities

#### 2.2.1. Calculating Probability from Frequencies

Probabilities of ngrams can be computed *normalizing* frequency counts (*Maximum Likelihood Estimation*): dividing the frequency of an ngram sequence by the frequency of its prefix (*relative frequency*).

N-gram   | Equation                      
:--------|:------------------------------
Unigram  | $$p(w_i) = \frac{c(w_i)}{N}$$ 
Bigram   | $$p(w_i | w_{i-1}) = \frac{c(w_{i-1}, w_i)}{c(w_{i-1})}$$ 
Ngram    | $$p(w_i | w_{i-n+1}^{i-1}) = \frac{c(w_{i-n+1}^{i-1}, w_i)}{c(w_{i-n+1}^{i-1})}$$ 

where:
- $N$ is the total number of words in a corpus
- $c(x)$ is the count of occurrences of $x$ in a corpus (x could be unigram, bigram, etc.)

##### Exercise

- define a function to compute ngram probabilities using ngram counts

- compute probabilities of ngrams ($n=2$) in the training set of NL2SparQL4NLU
- report probabilities of the following ngrams
    - $p(the | of)$
    - $p(the | is)$
    - $p(play | the)$
    - probabilities of all bigram where $w_1$ is "*italy*", i.e. $p(*|italy)$


| ngram            | $$\approx p$$ | $$\approx\log(p)$$ |
|:-----------------|--------------:|-------------------:|
| $$p(the|of)$$    | 0.31          | -1.18     
| $$p(the|is)$$    | 0.36          | -1.03
| $$p(play|the)$$  | 0.00          | 
| $$p(make|italy)$$| 0.50          |
| $$p(in|italy)$$  | 0.50          |


#### 2.2.2. Underflow Problem

Probabilities are usually small ($<1$).
Multiplying many of those may cause *underflow* problem.

Use the sum of the probabilities' logs instead of product

| Properties     
|:---------------
| $$p(a) > p(b)$$
| $$log(p(a)) > log(p(b))$$
| $$log(a*b) = log(a) + log(b)$$
| $$p(a) * p(b) \rightarrow log(p(a)) + log(p(b))$$


##### Exercise
- update the function to compute probabilities to use log (use `math` library)
- define a function to convert log probabilities to probabilities (`exp()`)

## 3. Using Ngram Language Models

A statistical [language model](https://en.wikipedia.org/wiki/Language_model) is a probability distribution over sequences of words. Given such a sequence, say of length $n$, it assigns a probability $p(w_{1},\ldots ,w_{n})$ ($p(w_{1}^{n})$, for compactness) to the whole sequence (using Chain Rule). Consequently, the unigram and bigram probabilities computed above constitute an ngram language model of our corpus.

It is more useful for Natural Language Processing to have a __probability__ of a sequence being legal, rather than a grammar's __boolean__ decision whether it is legal or not.

### 3.1. Computing Probability of a Sequence (Scoring)

The most common usage of a language model is to compute probability of a sequence.

#### 3.1.1. Probability of a Sequence: [Chain Rule](https://en.wikipedia.org/wiki/Chain_rule_(probability))

Probability of a sequence is computed as a product of conditional probabilities (chain rule). 

$$p(w_{1}^{n}) = p(w_1) p(w_2|w_1) p(w_3|w_1^2) ... p(w_n|w_{1}^{n-1}) = \prod_{i=1}^{n}{p(w_i|w_{1}^{i-1})}$$

The order of ngram makes a simplifying assumption that probability of a current word only depends on previous $N - 1$ elements. Thus, it truncates previous context (history) to length $N - 1$.

$$p(w_i|w_{1}^{i-1}) \approx p(w_i|w_{i-N+1}^{i-1})$$

Consequently we have:

N-gram   | Equation                   |
:--------|:---------------------------|
unigram  | $$p(w_i)$$                 |
bigram   | $$p(w_i|w_{i-1})$$         |
trigram  | $$p(w_i|w_{i-2},w_{i-1})$$ |

The probability of the whole sequence applying an ngram model becomes:

$$p(w_{1}^{n}) = \prod_{i=1}^{n}{p(w_i|w_{i-N+1}^{i-1})}$$

#### 3.1.2. Sentence beginning & end tags

Including sentence boundary markers leads to a better model. To do that we need to augment each sentence with a special symbols for beginning and end of sentence tags (`<s>` and `</s>`, respectively). The beginning of the sentence tag gives the bigram context of the first word; and encodes probability of a word to start a sentence. Adding the end of the sentence tag, on the other hand, makes the bigram model a true probability distribution (Jurafsky and Martin). "Without it, the sentence probabilities for all sentences of a given length would sum to one. This model would define an infinite set of probability distributions, with one distribution per sentence length."

For larger ngrams, we’ll need to assume extra context for the contexts to the left and right of the sentence boundaries. For example, to compute trigram probabilities at the very beginning of the sentence, we can use two pseudo-words for the first trigram (i.e. `['<s>', '<s>', w1]`). Alternatively, we can use [back-off](https://en.wikipedia.org/wiki/Katz%27s_back-off_model), and use the `['<s>', w1]` bigram probability. 

__Example__: "The cat is fat"

$p($ `<s>`, the, cat, is, fat, `</s>` $) = p(the |$ `<s>` $) * p(cat | the) * p(is | cat) * p(fat | is) * p($ `</s>` $| fat)$


##### Exercise
- define a function to add sentence beginning and end tags to a corpus as list-of-lists
- define a function to compute sentence probability given an ngram model
    - remember that for log we use sum for raw probabilities product
        - use `math.prod` for Python 3.8
        - use `numpy.prod` for Python < 3.8
- re-compute bigram probabilities for the training set of NL2SparQL4NLU
- compute probability of sentences: *star of twilight* and *star of thor*

### 3.2. Generating Sequences

Ngram Model can be used as an automaton to generate probable legal sequences using the algorithm below.

__Algorithm for Bigram LM__

- $w_{i-1} = $ `<s>`;
- *while* $w_i \neq $ `</s>`

    - stochastically get new word w.r.t. $p(w_i|w_{i-1})$

#### Exercise
- define a function to generate random sentences (e.g. using `random.choice`)
- generate `5` different sentences & score them

### 3.3. Evaluation: [Perplexity](https://en.wikipedia.org/wiki/Perplexity)

- Measures how well model fits test data
- Probability of test data
- Weighted average branching factor in predicting the next word (lower is better).
- Computed as:

$$ PPL = \sqrt[N]{\frac{1}{p(w_1,w_2,...,w_N)}} = \sqrt[N]{\frac{1}{\prod_{i=1}^{N}p(w_i|w_{i-N+1})}}$$

Where $N$ is the number of words in test set.

### 3.4. Handling Unseen Words

Out-Of-Vocabulary (OOV) word -- tokens in test data that are not contained in the lexicon (vocabulary).
Empirically each OOV word results in 1.5 - 2 extra errors (> 1 due to the loss of contextual information).

__*How to handle words (in test set) that were never seen in the training data?*__

Train a language model with specific token (e.g. `<unk>`) for unknown words!

__*How to estimate probabilities of unknown words and ngrams?*__

The *simplest* approach is to replace all the words that are not in vocabulary (lexicon) with the `<unk>` token and treat it as any other word. (For instance, applying frequency cut-off to the lexicon, will allow estimate these probabilities on the training set.)

#### Exercise
- define a function to replace OOV words in a corpus as list-of-list given a lexicon
- re-compute bigram probabilities for the training set of NL2SparQL4NLU (with `<unk>`)
- re-compute probability of sentences: *star of twilight* and *star of thor*
    - replace unknown words and add tags as well
- compare scores to the ones without unknown word handling

### 3.6. Handling Data Sparseness

What do we do with words that are in our lexicon, but appear in a test set in an unseen context (i.e. no ngram)?

Similar to unseen words and unseen n-grams have $0$ probability; thus, whole sequence gets $0$ probability.
(The problem is somewhat avoided using log probabilities.)

Use smoothing:
- Add some probability to unseen events
- Remove some probability from seen events
- Joint probability distribution sums to 1!

##### Smoothing Methods

Available smoothing methods: ([tutorial](https://nlp.stanford.edu/~wcmac/papers/20050421-smoothing-tutorial.pdf))
- [Additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing) (__simplest__)
- Good-Turing estimate
- Jelinek-Mercer smoothing (interpolation)
- Katz smoothing (backoff)
- Witten-Bell smoothing
- Absolute discounting
- Kneser-Ney smoothing


#### 3.6.1. Add-One Smoothing
Kind of Additive Smoothing.

__Bigrams__

$V$ -- vocabulary size

$$p(w_i | w_{i-1}) = \frac{c(w_{i-1},w_i)+1}{c(w_{i-1})+V}$$

__N-grams__

$V$ -- total number of possible $(N-1)$-grams

$$p(w_i | w^{i-1}_{i-N+1}) = \frac{c(w^{i-1}_{i-N+1},w_i)+1}{c(w^{i-1}_{i-N+1})+V}$$

Typically, we assume $V = \{w : c(w) > 0\} \cup \{$ `<unk>` $\}$


#### Exercise
- update the ngram probability calculation function to apply add one smoothing
- re-compute bigram probabilities for the training set of NL2SparQL4NLU with smoothing
- compare scores to the ones without smoothing

#### 3.6.2. Interpolation (Jelinek-Mercer Smoothing)
[Interpolation](https://en.wikipedia.org/wiki/Interpolation) estimates the probability by combining more robust, but weaker estimators; i.e. fall-back to lower order ngram probabilities to estimate a higher ngram probability. 
The standard way of combination is to use *linear interpolation*: to estimate a probability of a trigram, we use a weighted sum of unigrams, bigrams, and trigram probabilities as:

$$p(w_i|w_{i-1}, w_{i-2}) = \lambda_3 p(w_i|w_{i-1},w_{i-2}) + \lambda_2 p(w_i|w_{i-1}) + \lambda_1 p(w_i)$$

Where $\lambda_1 + \lambda_2 + \lambda_3 = 1$.

> For any size of ngram: $n^{th}$-order smoothed model is defined recursively as a linear interpolation between the $n^{th}$-order Maximum Likelihood (ML) model and the $(n−1)^{th}$-order smoothed model.

$$p_{INT}(w_i|w_{i-N+1}^{i-1}) = \lambda_{w_{i-N+1}^{i-1}} p_{ML}(w_i|w_{i-N+1}^{i-1}) + (1-\lambda_{w_{i-N+1}^{i-1}}) p_{INT}(w_i|w_{i-N+2}^{i-1})$$

The recursion can be grounded as:
- unigram model: $p_{ML}(w_i)$
- uniform distribution (e.g. for OOV)

$$p_{U}(w_i)=\frac{1}{V}$$

Values of $\lambda$s are computed using *__deleted interpolation__*: 

> we successively delete each trigram from the training corpus and choose the $\lambda$s so as to maximize the likelihood of the rest of the corpus (similar to leave-one-out cross-validation). 

The deletion helps to set the $\lambda$s in a way to prevent over-fitting.

In [ ]:
def deleted_interpolation(counts):
    """
    generic deleted interpolation
    :param counts: counts trie
    :return: interpolation weights for ngram models
    """
    w = [0] * counts.size
    for ngram in counts.traverse():
        # current ngram count
        v = counts.get(ngram).count
        # (n)-gram counts
        n = [counts.get(ngram[0:i+1]).count for i in range(len(ngram))]
        # (n-1)-gram counts -- parent node
        p = [counts.get(ngram[0:i]).count for i in range(len(ngram))]
        # -1 from both counts & normalize
        d = [float((n[i]-1)/(p[i]-1)) if (p[i]-1 > 0) else 0.0 for i in range(len(n))]
        # increment weight of the max by raw ngram count
        k = d.index(max(d))
        w[k] += v
    return [float(v)/sum(w) for v in w]

### 3.7. Putting it all together (Exercise)

Train an Ngram Language Model (compute ngram probabilities) such that:

- case insensitive (by default)
- 2-gram
- log probabilities
- considers sentence boundaries (beginning and end of sentence tags)
- considers unknown words
- Add-One Smoothing

Compute probabilities of utterances in `NL2SparQL4NLU/dataset/NL2SparQL4NLU.test.utterances.txt`